In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import os

In [6]:
# Cargar datos
datos = pd.read_excel('datos_viento.xlsx', sheet_name='waves P2 P6')

# Extraer mes de la fecha
datos['Mes'] = datos['Fecha y Hora'].dt.month

# Función para convertir dirección en grados a rosa de los vientos
def degToCompass(num, cant=8):
    n = 360 / cant
    l = int(8 / cant)
    val = int((num / n) + .5)
    arr = ["N", "NE", "E", "SE", "S", "SO", "O", "NO"]
    arr = arr[::l]
    return arr[(val % cant)]

# Convertir direcciones a rosa de los vientos
datos['P2_Dir_Rosa'] = datos['P2_Dir'].apply(lambda x: degToCompass(x, 8))
datos['P6_Dir_Rosa'] = datos['P6_Dir'].apply(lambda x: degToCompass(x, 8))

In [10]:
# Definir orden específico para meses y direcciones del viento
mes_order = list(range(1, 13))
windrose_order = ["N", "NE", "E", "SE", "S", "SO", "O", "NO"]

# Convertir direcciones y meses a categorías ordenadas
datos['Mes'] = pd.Categorical(datos['Mes'], categories=mes_order, ordered=True)
datos['P2_Dir_Rosa'] = pd.Categorical(datos['P2_Dir_Rosa'], categories=windrose_order, ordered=True)
datos['P6_Dir_Rosa'] = pd.Categorical(datos['P6_Dir_Rosa'], categories=windrose_order, ordered=True)

# Definir bins para altura, período y otras variables
hs_bins = pd.interval_range(start=0, end=10, freq=0.5)
tp_bins = pd.interval_range(start=0, end=20, freq=1)
tm01_bins = pd.interval_range(start=0, end=20, freq=1)
tz_bins = pd.interval_range(start=0, end=20, freq=1)
tmax_bins = pd.interval_range(start=0, end=20, freq=1)
hmax_bins = pd.interval_range(start=0, end=10, freq=0.5)

# Crear bins para variables relevantes
datos['P2_Hs_bins'] = pd.cut(datos['P2_Hs'], bins=hs_bins)
datos['P6_Hs_bins'] = pd.cut(datos['P6_Hs'], bins=hs_bins)
datos['P2_Tp_bins'] = pd.cut(datos['P2_Tp'], bins=tp_bins)
datos['P6_Tp_bins'] = pd.cut(datos['P6_Tp'], bins=tp_bins)
datos['P2_Tm01_bins'] = pd.cut(datos['P2_Tm01'], bins=tm01_bins)
datos['P6_Tm01_bins'] = pd.cut(datos['P6_Tm01'], bins=tm01_bins)
datos['P2_Tz_bins'] = pd.cut(datos['P2_Tz'], bins=tz_bins)
datos['P6_Tz_bins'] = pd.cut(datos['P6_Tz'], bins=tz_bins)
datos['P2_Tmax_bins'] = pd.cut(datos['P2_Tmax'], bins=tmax_bins)
datos['P6_Tmax_bins'] = pd.cut(datos['P6_Tmax'], bins=tmax_bins)
datos['Hmax_P2_bins'] = pd.cut(datos['Hmax_P2'], bins=hmax_bins)
datos['Hmax_P6_bins'] = pd.cut(datos['Hmax_P6'], bins=hmax_bins)

# Función para crear tablas de ocurrencia usando pd.crosstab
def create_occurrence_table(data, var1, var2):
    table = pd.crosstab(data[var1], data[var2], margins=True, margins_name='Suma Total', normalize='all') * 100
    table = table.round(2)
    return table

# Crear un colormap continuo personalizado
custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', ['white', 'green'])

# Función para graficar la tabla de ocurrencia y guardarla como archivo PNG
def plot_occurrence_heatmap(table, title, xlabel, ylabel, filename, vmin=0, vmax=15, cmap='viridis'):
    plt.figure(figsize=(12, 8))
    # Mascara para los valores cero
    mask = table.iloc[:-1, :-1] == 0
    # Dibujar el heatmap sin colorbar
    sns.heatmap(table.iloc[:-1, :-1], annot=True, fmt=".2f", cmap=custom_cmap, vmin=vmin, vmax=vmax, annot_kws={"size": 12}, mask=mask, cbar=False, linewidths=.5, linecolor='grey')
    plt.title(title, fontsize=16)
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.xticks(fontsize=12, rotation=45, ha='left')
    plt.yticks(fontsize=12)
    plt.gca().xaxis.set_ticks_position('top')  # Mueve los xticks hacia la parte superior
    plt.gca().xaxis.set_label_position('top')  # Mueve la etiqueta del eje X hacia la parte superior
    plt.tight_layout()
    plt.savefig(filename, format='png')
    plt.close()

# Listas de combinaciones para P2 y P6 con etiquetas descriptivas
combinations_P2 = [
    ('P2_Hs_bins', 'P2_Dir_Rosa', 'Distribución de Altura Significativa (Hs) por Dirección del Viento (Punto 2)', 'Dirección del Viento', 'Altura Significativa (Hs) [m]', 'P2_Hs_vs_Dir_Rosa.png'),
    ('P2_Hs_bins', 'Mes', 'Distribución de Altura Significativa (Hs) por Mes (Punto 2)', 'Mes', 'Altura Significativa (Hs) [m]', 'P2_Hs_vs_Mes.png'),
    ('P2_Hs_bins', 'P2_Tp_bins', 'Distribución de Altura Significativa (Hs) por Período Pico (Tp) (Punto 2)', 'Período Pico (Tp) [s]', 'Altura Significativa (Hs) [m]', 'P2_Hs_vs_Tp.png'),
    ('P2_Tp_bins', 'P2_Dir_Rosa', 'Distribución de Período Pico (Tp) por Dirección del Viento (Punto 2)', 'Dirección del Viento', 'Período Pico (Tp) [s]', 'P2_Tp_vs_Dir_Rosa.png'),
    ('P2_Tp_bins', 'P2_Tm01_bins', 'Distribución de Período Pico (Tp) por Período Medio (Tm01) (Punto 2)', 'Período Medio (Tm01) [s]', 'Período Pico (Tp) [s]', 'P2_Tp_vs_Tm01.png'),
    ('P2_Tp_bins', 'P2_Tz_bins', 'Distribución de Período Pico (Tp) por Período de Cruce (Tz) (Punto 2)', 'Período de Cruce (Tz) [s]', 'Período Pico (Tp) [s]', 'P2_Tp_vs_Tz.png'),
    ('Hmax_P2_bins', 'P2_Tmax_bins', 'Distribución de Altura Máxima (Hmax) por Período Máximo (Tmax) (Punto 2)', 'Período Máximo (Tmax) [s]', 'Altura Máxima (Hmax) [m]', 'Hmax_vs_Tmax_P2.png')
]

combinations_P6 = [
    ('P6_Hs_bins', 'P6_Dir_Rosa', 'Distribución de Altura Significativa (Hs) por Dirección del Viento (Punto 6)', 'Dirección del Viento', 'Altura Significativa (Hs) [m]', 'P6_Hs_vs_Dir_Rosa.png'),
    ('P6_Hs_bins', 'Mes', 'Distribución de Altura Significativa (Hs) por Mes (Punto 6)', 'Mes', 'Altura Significativa (Hs) [m]', 'P6_Hs_vs_Mes.png'),
    ('P6_Hs_bins', 'P6_Tp_bins', 'Distribución de Altura Significativa (Hs) por Período Pico (Tp) (Punto 6)', 'Período Pico (Tp) [s]', 'Altura Significativa (Hs) [m]', 'P6_Hs_vs_Tp.png'),
    ('P6_Tp_bins', 'P6_Dir_Rosa', 'Distribución de Período Pico (Tp) por Dirección del Viento (Punto 6)', 'Dirección del Viento', 'Período Pico (Tp) [s]', 'P6_Tp_vs_Dir_Rosa.png'),
    ('P6_Tp_bins', 'P6_Tm01_bins', 'Distribución de Período Pico (Tp) por Período Medio (Tm01) (Punto 6)', 'Período Medio (Tm01) [s]', 'Período Pico (Tp) [s]', 'P6_Tp_vs_Tm01.png'),
    ('P6_Tp_bins', 'P6_Tz_bins', 'Distribución de Período Pico (Tp) por Período de Cruce (Tz) (Punto 6)', 'Período de Cruce (Tz) [s]', 'Período Pico (Tp) [s]', 'P6_Tp_vs_Tz.png'),
    ('Hmax_P6_bins', 'P6_Tmax_bins', 'Distribución de Altura Máxima (Hmax) por Período Máximo (Tmax) (Punto 6)', 'Período Máximo (Tmax) [s]', 'Altura Máxima (Hmax) [m]', 'Hmax_vs_Tmax_P6.png')
]

# Crear carpeta para guardar los gráficos
output_dir = 'plots'
os.makedirs(output_dir, exist_ok=True)

# Generar y graficar tablas de ocurrencia para P2 y P6
for var1, var2, title, xlabel, ylabel, filename in combinations_P2:
    table = create_occurrence_table(datos, var1, var2)
    plot_occurrence_heatmap(table, title, xlabel, ylabel, os.path.join(output_dir, filename))

for var1, var2, title, xlabel, ylabel, filename in combinations_P6:
    table = create_occurrence_table(datos, var1, var2)
    plot_occurrence_heatmap(table, title, xlabel, ylabel, os.path.join(output_dir, filename))

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Cargar datos
datos = pd.read_excel('datos_viento.xlsx', sheet_name='waves P2 P6')

# Función para procesar datos y graficar modelos
def procesar_y_graficar(datos, punto):
    # Filtrar filas con valores no nulos en las columnas relevantes
    datos = datos[[f'{punto}_Hs', f'{punto}_Tp', f'{punto}_Tm01', f'{punto}_Tz', f'{punto}_Tmax', f'Hmax_{punto}']].dropna()

    # Separar variables independientes (X) y dependientes (y) para las dos relaciones
    X_Hs = datos[[f'{punto}_Hs']].values
    y_Tp = datos[f'{punto}_Tp'].values
    y_Hmax = datos[f'Hmax_{punto}'].values

    # Dividir datos en conjuntos de entrenamiento y prueba
    X_train_Hs, X_test_Hs, y_train_Tp, y_test_Tp = train_test_split(X_Hs, y_Tp, test_size=0.3, random_state=42)
    _, _, y_train_Hmax, y_test_Hmax = train_test_split(X_Hs, y_Hmax, test_size=0.3, random_state=42)

    # Función para ajustar y graficar modelos
    def ajustar_y_graficar_modelo(X_train, X_test, y_train, y_test, model, model_name, y_label, plot_name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        # Obtener coeficientes de la regresión lineal
        coef = model.coef_[0]
        intercept = model.intercept_
        eqn_text = f'y = {coef:.2f}x + {intercept:.2f}'
        
        plt.figure(figsize=(12, 8))
        plt.scatter(X_test, y_test, color='black', alpha=0.5, label='Datos Reales')
        plt.plot(X_test, y_pred, color='blue', linewidth=2, label=f'{eqn_text}\n(RMSE={rmse:.2f}, $R^2$={r2:.2f})')  # Add newline before equation
        plt.xlabel('Altura Significativa (Hs) [m]', fontsize=14)
        plt.ylabel(y_label, fontsize=14)
        plt.legend()
        plt.title(f'Relación {y_label}(Hs) - {model_name} ({punto})', fontsize=16)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.tight_layout()
        if not os.path.exists('rectas'):
            os.makedirs('rectas')
        plt.savefig(f'rectas/{plot_name}.png')  # Guardar la figura en la carpeta "rectas"
        plt.close()

    # Ajustar y graficar modelo de Regresión Lineal para Tp(Hs)
    ajustar_y_graficar_modelo(X_train_Hs, X_test_Hs, y_train_Tp, y_test_Tp, LinearRegression(), 'Regresión Lineal', 'Período Pico (Tp) [s]', f'{punto}_Tp_vs_Hs')

    # Ajustar y graficar modelo de Regresión Lineal para Hmax(Hs)
    ajustar_y_graficar_modelo(X_train_Hs, X_test_Hs, y_train_Hmax, y_test_Hmax, LinearRegression(), 'Regresión Lineal', 'Altura Máxima (Hmax) [m]', f'{punto}_Hmax_vs_Hs')



# Procesar y graficar datos para P2
procesar_y_graficar(datos, 'P2')

# Procesar y graficar datos para P6
procesar_y_graficar(datos, 'P6')